### LazyMIL: Automated MIL Benchmarking and Smart Consensus Modeling

The **LazyMIL** module provides a convenient, high-level interface for applying **Multiple Instance Learning (MIL)** to real-world or benchmark datasets.  
It seamlessly combines **descriptor calculation**, **model training**, and **evaluation** into one streamlined workflow — ideal for competitions, benchmarks, or quick exploratory studies.

LazyMIL automatically:
- Handles **descriptor calculation** for molecules or fragments.  
- Trains **multiple MIL estimators** in parallel.  
- Collects predictions and metrics for model comparison.  
- Optionally integrates **smart consensus optimization** using a genetic algorithm.

For consensus modeling, LazyMIL leverages the **QSARcons** package — a flexible framework for discovering optimal model ensembles.

> 🧩 **Install QSARcons before running this tutorial:**
> ```bash
> pip install qsarcons
> ```

**In summary:**  
LazyMIL simplifies the process of testing, comparing, and combining MIL models, making it a practical tool for QSAR researchers and ML competitions alike.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from qsarmil.lazy import LazyMIL
from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor

### 1. Loading ACE Dataset

ACE dataset

In [ ]:
dataset_url = "https://raw.githubusercontent.com/molML/MoleculeACE/main/MoleculeACE/Data/benchmark_data/CHEMBL2034_Ki.csv"
dataset = pd.read_csv(dataset_url)

data_train = dataset[dataset["split"] == "train"][["smiles", "y"]]
data_test = dataset[dataset["split"] == "test"][["smiles", "y"]]
data_train, data_val = train_test_split(data_train, test_size=0.2, random_state=42)

### 2. Build multiple MIL models models with conformers as instances

In [ ]:
lazy_mil = LazyMIL(task="regression", hopt=False, output_folder="ace_bench_default_2034", n_cpu=20, verbose=True)
lazy_mil.run(data_train, data_val, data_test)

### 3. Build model consensus

In [ ]:
metric = "auto"
cons_size = "auto"

cons_size_candidates = [2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
cons_methods = [
    ("Best", SystematicSearchRegressor(cons_size=1, cons_size_candidates=cons_size_candidates, metric=metric)),
    
    ("Random", RandomSearchRegressor(cons_size=cons_size, cons_size_candidates=cons_size_candidates, n_iter=1000, metric=metric)), 
    
    ("Systematic", SystematicSearchRegressor(cons_size=cons_size, cons_size_candidates=cons_size_candidates, metric=metric)),
    
    ("Genetic", GeneticSearchRegressor(cons_size=cons_size, n_iter=50, cons_size_candidates=cons_size_candidates, 
                                       pop_size=50, mut_prob=0.2, metric=metric))
]

In [ ]:
# load model predictions
df_val = pd.read_csv("ace_bench_default_2034/val.csv")
df_test = pd.read_csv("ace_bench_default_2034/test.csv")

# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test = df_test.iloc[:, 2:]

In [ ]:
for name, cons_searcher in cons_methods:

    # run search
    best_cons = cons_searcher.run(x_val, true_val)
    
    # make val and test predictions
    pred_val = cons_searcher._consensus_predict(x_val[best_cons])
    pred_test = cons_searcher._consensus_predict(x_test[best_cons])
    
    # write prediction accuracy metric
    df_val[name] = pred_val
    df_test[name] = pred_test

In [ ]:
best_cons

### 4. Summurize results

In [ ]:
res = pd.DataFrame()
for model in df_val.columns[2:]:
    res.loc[model, "R2"] = r2_score(df_val["Y_TRUE"], df_val[model])
res.sort_values(by="R2", ascending=False)

In [ ]:
res = pd.DataFrame()
for model in df_test.columns[2:]:
    res.loc[model, "R2"] = r2_score(df_test["Y_TRUE"], df_test[model])
res.sort_values(by="R2", ascending=False).round(2)